# Bayesian Networks

In this programming assignment, we will investigate the structure of the binarized MNIST dataset of handwritten digits using Bayesian networks. The dataset contains images of handwritten digits with dimensions $28 \times 28$ (784) pixels. Consider the Bayesian network in Figure 1 . The network contains two layers of variables. The variables in the bottom layer, $X_{1: 784}$ denote the pixel values of the flattened image. The variables in the top layer, $Z_{1}$ and $Z_{2}$, are referred to as latent variables, because the value of these variables will not be explicitly provided by the data and will have to be inferred.

![Figure1](./Images/fig1.png)

The Bayesian network specifies a joint probability distribution over binary images and latent variables $p\left(Z_{1}, Z_{2}, X_{1: 784}\right)$. The model is trained so that the marginal probability of the manifest variables, $p\left(x_{1: 784}\right)=\sum_{z_{1}, z_{2}} p\left(z_{1}, z_{2}, x_{1: 784}\right)$ is high on images that look like digits, and low for other images. 

For this programming assignment, we provide a pretrained model trained_mnist_model. The starter code loads this model and provides functions to directly access the conditional probability tables. Further, we simplify the problem by discretizing the latent and manifest variables such that $\operatorname{Val}\left(Z_{1}\right)=\operatorname{Val}\left(Z_{2}\right)=\{-3,-2.75, \ldots, 2.75,3\}$ and $\operatorname{Val}\left(X_{j}\right)=\{0,1\}$, i.e., the image is binary.

### 1. 
How many values can the random vector $X_{1: 784}$ take, i.e., how many different $28 \times 28$ binary images are there?

How many parameters would you need to specify an arbitrary probability distribution over all possible $28 \times 28$ binary images? (5 points)

In [ ]:
print('There can be at most 2^(28*28) different binary images')

In [ ]:
print('There can be at most 25*25*28*28 parameters')

Run below codes to load the network and its functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from scipy.io import loadmat
NUM_PIXELS = 28*28

In [ ]:
def get_p_z1(z1_val):
    '''
    Helper. Computes the prior probability for variable z1 to take value z1_val.
    P(Z1=z1_val)
    '''
    return bayes_net['prior_z1'][z1_val]
def get_p_z2(z2_val):
    '''
    Helper. Computes the prior probability for variable z2 to take value z2_val.
    P(Z2=z2_val)
    '''
    return bayes_net['prior_z2'][z2_val]

In [ ]:
def get_p_xk_cond_z1_z2(z1_val, z2_val, k):
    '''
    Helper. Computes the conditional probability that variable xk assumes value 1
    given that z1 assumes value z1_val and z2 assumes value z2_val
    P(Xk = 1 | Z1=z1_val , Z2=z2_val)
    '''
    return bayes_net['cond_likelihood'][(z1_val, z2_val)][0, k-1]


def get_p_x_cond_z1_z2(z1_val, z2_val):
    '''
    Computes the conditional probability of the entire vector x for x = 1,
    given that z1 assumes value z1_val and z2 assumes value z2_val
    '''
    pk = np.zeros(NUM_PIXELS)
    for i in range(NUM_PIXELS):
        pk[i] = get_p_xk_cond_z1_z2(z1_val, z2_val, i+1)
    return pk

In [ ]:
def load_model(model_file):
    '''
    Loads a default Bayesian network with latent variables (in this case, a
    variational autoencoder)
    '''
    with open('Helper_codes/trained_mnist_model', 'rb') as infile:
        cpts = pkl.load(infile, encoding='bytes')
    model = {}
    model['prior_z1'] = cpts[0]
    model['prior_z2'] = cpts[1]
    model['cond_likelihood'] = cpts[2]
    return model

In [ ]:
global disc_z1, disc_z2
n_disc_z = 25
disc_z1 = np.linspace(-3, 3, n_disc_z)
disc_z2 = np.linspace(-3, 3, n_disc_z)

global bayes_net
bayes_net = load_model('Helper_codes/trained_mnist_model')

### 2.
Produce 5 samples from the joint probability distribution $\left(z_{1}, z_{2}, x_{1: 784}\right) \sim p\left(Z_{1}, Z_{2}, X_{1: 784}\right)$, and plot the corresponding images (values of the pixel variables). (7 points)

In [ ]:
fig = plt.figure(figsize=(15, 15))
p = np.zeros(5)
for i in range(5):
    z1 = np.random.choice(disc_z1)
    z2 = np.random.choice(disc_z2)
    Conditional_Probability = get_p_x_cond_z1_z2(z1, z2)
    Vector_Sample = np.random.binomial(1, 0.5, 784)
    Matrix_Sample = np.reshape(Vector_Sample,[28,28])
    p[i] = get_p_z1(z1) * get_p_z2(z2)
    for j in range(784):
        if Vector_Sample[i] == 1:
            p[i] *= Conditional_Probability[i]
        else:
            p[i] *= (1-Conditional_Probability[i])
    fig.add_subplot(1, 5, i+1)
    plt.imshow(Matrix_Sample,cmap='gray', vmin=0, vmax=1)
plt.show()
for i in range(5):
    print('Joint probability distribution of sample %d is %.20f' % (i, p[i]))  

### 3.

For each possible value of
$$
\left(\bar{z}_{1}, \bar{z}_{2}\right) \in\{-3,-2.75, \ldots, 2.75,3\} \times\{-3,-2.75, \ldots, 2.75,3\}
$$
compute the conditional expectation $E\left[X_{1: 784} \mid Z_{1}, Z_{2}=\left(\bar{z}_{1}, \bar{z}_{2}\right)\right] .$ This is the expected image corresponding to each possible value of the latent variables $Z_{1}, Z_{2} .$ Plot the images on on a $2 \mathrm{D}$ grid where the grid axes correspond to $Z_{1}$ and $Z_{2}$ respectively. What is the intuitive role of the $Z_{1}, Z_{2}$ variables in this model? (8 points)

In [ ]:
fig, axs = plt.subplots(len(disc_z1), len(disc_z2),figsize=(20, 20))
for i in range(len(disc_z1)):
    z1 = disc_z1[i]
    for j in range(len(disc_z2)): 
        z2 = disc_z2[j]
        Conditional_Probability = get_p_x_cond_z1_z2(z1, z2)
        Matrix_Sample = np.reshape(Conditional_Probability,[28,28])
        axs[i, j].imshow(Matrix_Sample,cmap='gray', vmin=0, vmax=1)


In [ ]:
mat = loadmat('Helper_codes/testval.mat')
val_data = mat['val_x']
test_data = mat['test_x']

### 4.
You are given a validation and a test dataset. In the test dataset, some images are "real" handwritten digits, and some are anomalous (corrupted images). We would like to use our Bayesian network to distinguish real images from the anomalous ones. Intuitively, our Bayesian network should assign low probability to corrupted images and high probability to the real ones, and we can use this for classification. To do this, we first compute the average marginal log-likelihood,
$$
\log p\left(x_{1: 784}\right)=\log \sum_{z_{1}} \sum_{z_{2}} p\left(z_{1}, z_{2}, x_{1: 784}\right)
$$
on the validation dataset, and the standard deviation (again, standard deviation over the validation set). Consider a simple prediction rule where images with marginal log-likelihood, $\log p\left(x_{1: 784}\right)$, outside three standard deviations of the average marginal log-likelihood are classified as corrupted. Classify images in the test set as corrupted or real using this rule. Then plot a histogram of the marginal log-likelihood for the images classified as "real". Plot a separate histogram of the marginal log-likelihood for the images classified as "corrupted". (15 points)

In [ ]:
k1 = np.size(test_data,axis=0)
Marginal_Likelihood = np.zeros(k1)
for i in range(k1):
    Vector = test_data[i]
    for j1 in range(len(disc_z1)):
        z1 = disc_z1[j1]
        for j2 in range(len(disc_z2)):
            z2 = disc_z2[j]       
            Conditional_Probability = get_p_x_cond_z1_z2(z1, z2)
            p = np.log(get_p_z1(z1)) + np.log(get_p_z2(z2))
            for k in range(784):
                if Vector[k] == 1:
                    p +=  np.log(Conditional_Probability[k])
                else:
                    p +=  np.log((1-Conditional_Probability[k]))
            Marginal_Likelihood[i] += p
            
My_Mean = np.mean(Marginal_Likelihood)
My_Std  = np.std(Marginal_Likelihood)

print(My_Mean,My_Std)

In [ ]:
k2 = np.size(val_data,axis=0)
Val_labels = np.zeros(k2)
Marginal_Likelihood_Val = np.zeros(k2)
Real_Image_Marginal_Likelihood = []
Corrupted_Image_Marginal_Likelihood = []
for i in range(k2):    
    Vector = val_data[i]
    for j1 in range(len(disc_z1)):
        z1 = disc_z1[j1]
        for j2 in range(len(disc_z2)):
            z2 = disc_z2[j]  
            Conditional_Probability = get_p_x_cond_z1_z2(z1, z2)
            p = np.log(get_p_z1(z1)) + np.log(get_p_z2(z2))
            for k in range(784):
                if Vector[k] == 1:
                    p += np.log(Conditional_Probability[k])
                else:
                    p += np.log((1-Conditional_Probability[k]))
            Marginal_Likelihood_Val[i] += p
            
    print(Marginal_Likelihood_Val[i], np.abs(Marginal_Likelihood_Val[i]-My_Mean), 3 * My_Std)
    if np.abs(Marginal_Likelihood_Val[i]-My_Mean) > 3 * My_Std:
        Val_labels[i] = 0
        Corrupted_Image_Marginal_Likelihood.append(Marginal_Likelihood_Val[i])
    else:
        Val_labels[i] = 1
        Real_Image_Marginal_Likelihood.append(Marginal_Likelihood_Val[i])

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
axs[0].hist(Corrupted_Image_Marginal_Likelihood, density=True)
axs[1].hist(Real_Image_Marginal_Likelihood, density=True)